In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
from typing import Dict, List, Tuple

np.set_printoptions(linewidth=300)

In [ ]:
baseDir = '/home/user/Schreibtisch/results/4-level qubit/two transmons'


def getSubdirectories(directory: str):
    return [name for name in os.listdir(directory) if os.path.isdir(os.path.join(directory, name))]


def cutByFidelity(fidelities: np.array, otherValues: np.array) -> Tuple[np.array, np.array]:
    indices = np.where(fidelities < 0.9)
    if len(indices[0]) > 0:
        last = indices[0][-1]
        fidelities = fidelities[last+1:]
        otherValues = otherValues[last+1:]
    return fidelities, otherValues


def cutByFrequency(frequencies: np.array, otherValues: np.array, maxFrequencies=None, minFrequencies=None) -> Tuple[np.array, np.array]:
    if maxFrequencies:
        indices = np.where(frequencies <= maxFrequencies)[0]
        if len(indices) > 0:
            frequencies = frequencies[:indices[-1]+1]
            otherValues = otherValues[:indices[-1]+1]
    if minFrequencies:
        indices = np.where(frequencies >= minFrequencies)[0]
        if len(indices) > 0:
            frequencies = frequencies[indices[0]:]
            otherValues = otherValues[indices[0]:]
    return frequencies, otherValues


def readFidelity(gateDir) -> float:
    subDir = [x[0] for x in os.walk(gateDir)][2]
    file = subDir + "/best_point_open_loop.c3log"
    with open(file, 'r') as file:
        line = [l for l in file if "goal:" in l][0]
        line = line.strip()[6:]
        return float(line)


def readFidelitiesForFrequencies(time: int, gate: str) -> Tuple[np.array, np.array]:
    gateBaseDir = f"{baseDir}/Fourier basis 5 levels, {time}ns/{gate}"
    subdirs = getSubdirectories(gateBaseDir)
    numbers = [int(s) for s in subdirs if s.isnumeric()]
    numbers.sort()
    frequencies = []
    fidelities = []
    for N in numbers:
        gateDir = f"{gateBaseDir}/{N}"
        if os.path.exists(gateDir):
            frequencies.append(N)
            fidelities.append(1.0 - readFidelity(gateDir))
    return np.array(frequencies), np.array(fidelities)


def readFidelitiesForTimes(frequencies: int, gateName: str, gateForTime: Dict[int, str]) -> Tuple[np.array, np.array]:
    times = []
    fidelities = []
    for time in [1000, 500, 400, 300, 200]:
        gate = gateForTime[time] if time in gateForTime else gateName
        gateDir = f"{baseDir}/Fourier basis 5 levels, {time}ns/{gate}/{frequencies}"
        if os.path.exists(gateDir):
            times.append(time)
            fidelities.append(1.0 - readFidelity(gateDir))
    return np.array(times), np.array(fidelities)

#readFidelitiesForFrequencies(1000, "unity")

## Fidelities vs number of frequencies for a fixed gate time

In [ ]:
def plotFidelityOverFrequenciesFixedT(T: int, gates: Dict[str, str], filename: str = None):
    fig = plt.figure(figsize=(10, 7))
    axis = fig.add_subplot(111)

    minFrequencies, maxFrequencies = 1000, 0
    markers = ['o', 'x', 's', 'd', 'o']
    for idx, (gateName, gateDir) in enumerate(gates.items()):
        frequencies, fidelities = readFidelitiesForFrequencies(T, gateDir)
        frequencies, fidelities = cutByFrequency(frequencies, otherValues=fidelities, maxFrequencies=200)
        fidelities, frequencies = cutByFidelity(fidelities, frequencies)
        plt.plot(frequencies, fidelities, label=gateName, marker=markers[idx], ms=7)
        minFrequencies = min(minFrequencies, min(frequencies))
        maxFrequencies = max(maxFrequencies, max(frequencies))

    plt.xlabel('Number of frequencies $N$', fontsize=24)
    plt.xscale('log')
    plt.xticks([x for x in [10, 15, 20, 30, 40, 50, 60, 80, 100, 200] if minFrequencies-5 <= x <= maxFrequencies+5])
    axis.get_xaxis().set_major_formatter(mpl.ticker.ScalarFormatter())

    plt.ylabel('Fidelity $F$', fontsize=24)
    #plt.yscale('log')
    #plt.yticks([x for x in [0.9, 0.92, 0.94, 0.96, 0.98, 1.0]])
    #plt.title(f"T = {T} ns", fontsize=20)

    axis.tick_params(axis='both', which='major', labelsize=24)
    #plt.hlines(0.9, minFrequencies, maxFrequencies, linestyles=":", colors="gray")
    plt.legend(loc='lower right', fontsize=24)
    plt.tight_layout()

    #fig.gca().get_xaxis().set_major_formatter(mpl.ticker.ScalarFormatter())
    if filename:
        plt.savefig(filename)
    plt.show()

#"unity": "unity", "CNOT": "CNOT", "iSWAP": "iSWAP"
plotFidelityOverFrequenciesFixedT(1000, {"CZ": "CZ neu", "CX": "CNOT", "CCCZ": "CCCZ", "CCCX": "CCCX"}, filename='/home/user/four_qubit_gates_fidelities.pdf')
#plotFidelityOverFrequenciesFixedT(500, {"unity": "unity", "CZ": "CZ", "CCCX": "CCCX"})
#plotFidelityOverFrequenciesFixedT(300, {"unity": "unity", "CZ": "CZ", "CCCX": "CCCX"})print("nach max: ", frequencies)

## Fidelities vs number of frequencies for a fixed gate

In [ ]:
def plotFidelityOverFrequenciesFixedGate(gateForTime: Dict[int, str], filename: str = None):
    fig = plt.figure(figsize=(10,7))
    axis = fig.add_subplot(111)

    minFrequencies, maxFrequencies = 1000, 0
    markers = ['o', 'x', 's', 'd']
    for idx, (T, gate) in enumerate(gateForTime.items()):
        frequencies, fidelities = readFidelitiesForFrequencies(T, gate)
        fidelities, frequencies = cutByFidelity(fidelities, frequencies)
        if len(fidelities) > 0:
            plt.plot(frequencies, fidelities, label=f"{T} ns", marker=markers[idx], ms=7)
            minFrequencies = min(minFrequencies, min(frequencies))
            maxFrequencies = max(maxFrequencies, max(frequencies))

    plt.xlabel('Number of frequencies $N$', fontsize=24)
    plt.xscale('log')
    plt.xticks([x for x in [10, 15, 20, 30, 40, 50, 70, 100, 200, 300, 500] if minFrequencies-5 <= x <= maxFrequencies+5])
    axis.get_xaxis().set_major_formatter(mpl.ticker.ScalarFormatter())

    plt.ylabel('Fidelity $F$', fontsize=24)
    #plt.title(gateName, fontsize=20)
    #plt.ylim(0.87, 1.01)

    axis.tick_params(axis='both', which='major', labelsize=24)
    #plt.hlines(0.9, minFrequencies, maxFrequencies, linestyles=":", colors="gray")
    plt.legend(loc='lower right', fontsize=24)
    plt.tight_layout()

    #fig.gca().get_xaxis().set_major_formatter(mpl.ticker.ScalarFormatter())
    if filename:
        plt.savefig(filename)
    plt.show()

#plotFidelityOverFrequenciesFixedGate("unity", {1000: "unity x-0", 500: "unity", 300: "unity", 200: "unity"})
plotFidelityOverFrequenciesFixedGate({1000: "CZ neu", 500: "CZ", 300: "CZ", 200: "CZ"}, filename='/home/user/two_transmons_fidelities_CZ.pdf')
plotFidelityOverFrequenciesFixedGate({1000: "CCCX", 500: "CCCX", 300: "CCCX", 200: "CCCX"}, filename='/home/user/two_transmons_fidelities_CCCX.pdf')

## Fidelity vs gate time for a fixed gate

In [ ]:
def plotFidelityOverTimeFixedGate(gateName: str, gateForTimes: Dict[int, str], frequencies: List[int]):
    fig = plt.figure(figsize=(10,8))
    axis = fig.add_subplot(111)

    allTimes = np.array([])
    for freq in frequencies:
        times, fidelities = readFidelitiesForTimes(freq, gateName, gateForTimes)
        plt.plot(np.array(times), fidelities, 'o-', label=f"{freq} frequencies")
        allTimes = np.append(allTimes, times)

    plt.xlabel('Gate time [ns]', fontsize=20)
    #plt.xscale('log')
    plt.xticks(list(set(allTimes)))
    axis.get_xaxis().set_major_formatter(mpl.ticker.ScalarFormatter())

    plt.ylabel('Fidelity', fontsize=20)
    plt.title(gateName, fontsize=20)
    #plt.ylim(0.87, 1.01)

    axis.tick_params(axis='both', which='major', labelsize=16)
    #plt.hlines(0.9, minFrequencies, maxFrequencies, linestyles=":", colors="gray")
    plt.legend(loc='lower right', fontsize=16)

    #fig.gca().get_xaxis().set_major_formatter(mpl.ticker.ScalarFormatter())
    plt.show()

plotFidelityOverTimeFixedGate("unity", {}, [100, 80, 60, 40, 35])
plotFidelityOverTimeFixedGate("CZ", {1000:"CZ from unity"}, [100, 80, 60, 40, 35])
plotFidelityOverTimeFixedGate("CCCZ", {}, [200, 100, 60, 60, 40, 30])
plotFidelityOverTimeFixedGate("CCCX", {}, [200, 100, 60, 60, 40, 30])
#plotFidelityOverTimeFixedGate("CZ", [100, 80, 60, 40, 35])